In [21]:
# import packages
from ast import literal_eval

# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
# from matplotlib import colors
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

# os and file types
import os
import sys
import datetime as dt
import json
import csv

In [22]:

# load the invasive species data from infoflora
invasive = pd.read_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\data\\species_lists\\inv.csv", encoding = 'latin1',sep = ';')

# drop 4 empty rows of nan values at the end of the data set
invasive= invasive.iloc[:-4]

# turn threat columns in invasive db into more useable framework. The resulting column is a list containing the threats specified in mythreats.
# note result are strings of integers, not integers!

mythreats = ["Potentiel d'expansion",'écologie, biodiversité']

mylist = []

for i in invasive.index:
    mysublist = []
    for j in mythreats:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append([j,invasive.at[i,j]])
    mylist.append(mysublist)
    for k in mylist:
        for l in k:
            if "xxx" in l[1]:
                l[1] = "3"
            if "xx" in l[1]:
                l[1] = "2"
            if "x" in l[1]:
                l[1] = "1"
            if "o" in l[1]:
                l[1] = '0'
invasive["threats"] = mylist

# turn location of invasives columns from db into more useable framework. The resulting column is a list containing the status of the places specified in myplaces.
# note result are strings of integers, not integers!

myplaces = ['Jura', 'Plateau','non établi en Suisse']

mylist = []

for i in invasive.index:
    mysublist = []
    for j in myplaces:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append([j,invasive.at[i,j]])
    mylist.append(mysublist)
    for k in mylist:
        for l in k:
            if "xxx" in l[1]:
                l[1] = "3"
            if "xx" in l[1]:
                l[1] = "2"
            if "x" in l[1]:
                l[1] = "1"
            if "o" in l[1]:
                l[1] = '0'
invasive["places"] = mylist

# collect all of the habitat data into one column. 
#In the db the presence of a species in each habitat is provided as a checkmark in the relevant column.
# For each species in the db, this code makes a list of the column names with a checkmark, and stores this data under a new "habitat" column

myhabitats = ['1 Eaux libres', '2 Rivages et lieu humides',
          '3 Glaciers, rochers, éboulis et moraines', '4 Pelouses et prairies',
          '5 Landes, lisières et mégaphorbiaies', '6 Forêts',
          '7 Végétations pionnières des endroits perturbés',
          '8 Plantations, champs, cultures', '9 Milieux construits']
mylist = []
for i in invasive.index:
    mysublist = []
    for j in myhabitats:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append(j)
    mylist.append(mysublist)
invasive["habitat"] = mylist

# keep select columns and write result to csv
keep_invasives = ['Latein','Black List / Watch List 2014','Ordonnonce sur la dissémination des organismes (ODE)', 'threats','places', 'habitat']
inv = invasive[keep_invasives].copy()



In [23]:
def to_slug(x):
    data = x.split()
    mydata = "".join(data[0:2]).lower()
    return mydata

In [24]:
inv["slug"] = inv.Latein.apply(lambda x: to_slug(x))

In [25]:
inv.to_csv("output/invasive_preproc.csv")

In [4]:
# load invasive_preproc file and transform list data stored as str in csv into lists for new df

inv = pd.read_csv('output/invasive_preproc.csv')
newinv = inv[['Latein', 'Black List / Watch List 2014','Ordonnonce sur la dissémination des organismes (ODE)']].copy()
columns = ['threats','places','habitat']
for i in columns:
    newinv[i] = inv[[i]].applymap(literal_eval)
inv = newinv

In [3]:
df = pd.read_csv('output/alldata.csv')

In [4]:
# keys to interpret invasive data
key_invasives_status = {'BL':'Black List', 'WL': 'Watch List', 'ODE': 'Prohibited'}
key_invasives_threat = {'1':'low impact','2':'medium impact','3':'high  impact'}
key_invasives_places = {'1':'low frequency','2':'medium frequency','3':'high frequency'}

In [5]:
inv.places

0       [[Jura, 1], [Plateau, 1]]
1       [[Jura, 2], [Plateau, 3]]
2       [[Jura, 3], [Plateau, 3]]
3       [[Jura, 1], [Plateau, 1]]
4       [[Jura, 2], [Plateau, 3]]
5       [[Jura, 1], [Plateau, 1]]
6       [[Jura, 3], [Plateau, 3]]
7       [[Jura, 3], [Plateau, 2]]
8     [[non établi en Suisse, 1]]
9     [[non établi en Suisse, 1]]
10      [[Jura, 1], [Plateau, 2]]
11    [[non établi en Suisse, 1]]
12      [[Jura, 3], [Plateau, 3]]
13      [[Jura, 1], [Plateau, 3]]
14      [[Jura, 3], [Plateau, 3]]
15      [[Jura, 3], [Plateau, 3]]
16    [[non établi en Suisse, 1]]
17      [[Jura, 3], [Plateau, 3]]
18                 [[Plateau, 2]]
19      [[Jura, 1], [Plateau, 2]]
20                 [[Plateau, 1]]
21    [[non établi en Suisse, 1]]
22      [[Jura, 1], [Plateau, 1]]
23      [[Jura, 1], [Plateau, 1]]
24      [[Jura, 1], [Plateau, 2]]
25      [[Jura, 2], [Plateau, 3]]
26      [[Jura, 1], [Plateau, 1]]
27                             []
28      [[Jura, 3], [Plateau, 3]]
29      [[Jura

In [6]:
# note that there are only two threat factors indicated for the plants here.
# the first threat factor 'potential d'expansion' varies little

mylist1 = []
mylist2 = []
for i in inv.threats:
    mylist1.append(int(i[0][1]))
    mylist2.append(int(i[1][1]))
 
inv['expansion'] = mylist1
inv['biodiverse'] = mylist2


In [33]:
mylist1 = []
mylist2 = []
for i in inv.places:
    if len(i) == 0:
        mylist1.append(0)
        mylist2.append(0)
    if len(i) == 1:
        print(i[0][0])
        if i[0][0] == "Jura":
            mylist1.append(i[0][1])
        if i[0][0] == "Plateau":
            mylist2.append(i[0][1])
        if i[0][0] == "non établi en Suisse":
            mylist1.append(0)
            mylist2.append(0)
        print(mylist1)
    if len(i) == 2:
        print(i[1][0])
        if i[1][0] == "Jura":
            mylist1.append(i[1][1])
        if i[1][0] == "Plateau":
            mylist2.append(i[1][1])
print(mylist1)
inv['plateau'] = mylist1
inv['jura'] = mylist2

Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0]
non établi en Suisse
[0, 0]
Plateau
non établi en Suisse
[0, 0, 0]
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0]
Plateau
Plateau
[0, 0, 0, 0]
Plateau
Plateau
[0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


ValueError: Length of values (11) does not match length of index (58)

In [ ]:
inv.expansion.mean()

In [ ]:


inv.biodiverse.mean()

In [ ]:
np.sqrt(inv.expansion.var())